<h1><b>Recuperação de Informação na Web 2020/2</b></h1>

<h3>Construção de um Coletor de Propósito Geral</h3>

Integrantes:    <ul><li>Antonio</li>
                <li>Mariana Bulgarelli Alves dos Santos</li>
                <li>Nathã Paulino</li></ul>

O coletor (crawler) busca realizar a coleta de dados de fontes diversas. Ele é composto de três partes principais: Downloader, Scheduler e Storage. O Downloader baixa cada uma das páginas enquanto o Scheduler mantém uma relação de páginas a serem requisitadas. No storage são armazenados os dados para possíveis  busca e indexação além de fornecer Metadados ao escalonador sobre as páginas baixadas.

Basicamente, o Crawler realiza o dowload de um grupo de sementes que são parseadas e visitadas para a coleta de novos links. Os links para páginas que não tiveram seu download realizado são armazenadas em uma fila para serem posteriormente coletadas.O crawler seleciona uma nova página para ser coletada e o processo segue até atingir o nível 3.

A arquitetura do crawler é apresentada a seguir:

<figure style="text-align:center">
    <img src="imgs/arquitetura.png">
    
    Fonte: [1]
</figure>

O coletor de URL's elaborado é composto de três classes principais, a saber:
<ul><li>Domain</li>
<li>Page_fetcher: threads que requisitam as URLs obtidas com o escalonador</li>
<li>Scheduler: armazena as filas de URLs a serem requisitadas</li></ul>


As URL's de interesse são armazenadas em uma fila, que no caso é um dicionário.


In [ ]:
Bibliotecas utilizadas:
from datetime import datetime
from collections import OrderedDict

from urllib import robotparser
from util.threads import synchronized
from collections import OrderedDict
from crawler.domain import Domain
from urllib.parse import urlparse, urlunparse
import time 
import urllib.robotparser
pandas

<p>A Classe <b>Domain</b> possui time_last_access, nam_domain (nome do domínio) e int_time_limit_seconds como atributos e armazena informações úteis para o momento da coleta, sendo que int_time_limit_seconds e nam_domain são passados como parâmetro.
O atributo int_time_limit_seconds indica de quanto em quanto tempo as requisições podem ser realizadas. O atributo time_last_access registra o momento do último aceeso à pagina.

Assim, ao tentar realizar uma requisição, primeiramente verifica-se se o servidor está acessível por meio da função is_accessible que verifica quando foi o último acesso comparando o retorno da função `time_since_last_access(self)` com `int_time_limit_seconds`. A time_since_last_access retorna um objeto TimeDelta com a diferença da data atual e a data do último acesso (time_last_access). Se o retorno for menor que `int_time_limit_seconds`, a função `is_accessible` retorna True. Assim, ao efetivar o acesso à página, a função accessed_now deve ser chamada e o atributo `time_last_access` modificado para o momento do acesso.

Além dessas funções, a classe Domain contém outras quatro funções que irão auxiliar na manipulação das urls. Como utilizamos um dicionário ordenado onde a key é um objeto da classe Domain e o value é uma lista de urls, para viabilizar a busca de modo direto pelo domínio utilizamos as funções `__hash__` (retorna o valor hash de um objeto, se houver - associa um valor de retorno para o objeto) e `__eq__`.

Os métodos `__str__` e `__repr__` retornam uma string que representa o objeto, no caso, nam_domain.
O tempo entre as requisições foi definido como 20segundos.</p>


<p> A Classe <b>Sheduler</b> trata-se do escalonador, que é responsável por armazenar e gerenciar a fila de urls. Confome explicado anteriormente, nossa fila de urls é na verdade um dicionário ordenado onde a key é um objeto da classe Domain (servidor) e value consiste em uma lista de tuplas de url e profundidade como objetos da classe parseResult (urlparse divide uma string de URL em seus componentes ou na combinação de componentes de URL em uma string de URL). Na coleta a profundidade foi limitada.
    
A Classe <b>Scheduler</b> possui como atributos `str_usr_agent` (nome do coletor - passado por parâmetro), `int_page_limit` (número de páginas a serem coletadas - passado por parâmetro), `int_depth_limit` (profundidade máxima a ser coletada - passado por parâmetro), `int_page_count` (quantidade de páginas já coletadas), `dic_url_per_domain` (OrderedDict() com a fila de URLs por domínio), `set_discovered_urls` (conjunto de URLs extraídas em algum HTML e já adicionadas na fila), `dic_robots_per_domain` (dicionário que contém o objeto com as regras do robots.txt para cada domínio) e `arr_urls_seeds` (array contendo urls, sendo que pra casa uma é criado um objeto urlparse e adicionado com profundidade de coleta 0 com `add_new_page`).
    
A função `can_add_page` verifica se a página não foi descoberta ainda e se a profundidade dela é menor que a profundidade limite e se as condicoes sao verdadeiras, retorna True. A função `get_next_url` obtem uma nova URL por meio da fila. Para isso, pega as keys do dicionario de urls e itera sobre elas verificando se o dominio esta acessivel (`is_accessible`). Se sim, é marcado como acessado, a url é removida da fila e fica em espera pelo tempo entre requisições.
    
A função `count_fetched_page` conta o número de paginas já coletadas e a `has_finished_crawl` verifica se a coleta foi finalizada, retornando true em caso verdadeiro.

A função `add_new_page` recebe como parâmetros um objeto da classe ParseResult com a URL a ser adicionada e a profundidade na qual foi coletada essa URL. Esta função adiciona uma nova página caso`can_add_page` retorne verdadeiro. Inicialmente é criado um objeto Domain com o netloc do objeto da classe ParseResult com a URL e com o tempo entre requisições para em seguida verificar se a página pode ser adicionada, retornando false se não puder. Assim, a url não é adicionada e o retorno é False. Caso contrário, adiciona-se o objeto como key do dicionario de urls por dominio com a profundidade de coleta como value e indica-se que essa página já foi descoberta.

Já a função `can_fetch_page` informa se uma url pode ou não ser coletada com base nas informações fornecidas pelo robots.txt.</p>

<h4>Bibliografia</h4>
[1] Dalip, D. H. Recuperação de Informação - Coletores na Web: arquitetura e política de escolha e "revisita" de páginas. Centro Federal de Educação Tecnológica de Minas Gerais, 2020.

In [ ]:
Conteúdo do Relatório


a) Principais desafios, decisões e arquitetura utilizada
b) URLs sementes utilizadas
c) Como foi feito, faça referencias à classes e métodos do código fonte:
→ Os critérios de exclusão de robôs e quantidade de tempo entre
requisições à um mesmo servidor

d) O impacto na velocidade de coleta (quantidade de páginas por segundo) ao
aumentar o número de threads 10 a 100, de 20 em 20



In [ ]:
e) Link para a página descrevendo o coletor criado
Não coloque muito código fonte no relatório – crie arquivo separado com o código
e, no relatório, apenas poucas linhas de código para gerar os gráficos/tabelas.

In [ ]:




Deverá ser entregue dia 07/10/2020. Com apresentações parciais (ver cronograma0.
A entrega deverá conter os seguintes itens em um arquivo comprimido:
a) Relatório do trabalho obrigatoriamente em Jupyter Notebook
b) Código fonte
c) Lista de URLs coletadas. Não é necessário salvar os arquivos.

Tarefas
Para fazer um coletor é necessário:
1) O escalonador deverá possuir:
(a) Uma fila de páginas, sendo coletado através de uma busca em largura
(b) Não permitir que a mesma url seja coletada mais de uma vez
(c) Armazenar a última vez que um servidor foi acessado. Pois, um servidor poderá
ser acessado de 30 em 30 segundos
2) Multiplas threads para coletar as páginas (os Page Fetchers):
(a) Coleta a página que o escalonador organizou
(b) Dado a página coletada, extrair seus links e inserir na fila do
escalonador todas as páginas coletadas
→ A classe ColetorUtil poderá auxiliar no caso de urls relativas, pois
os links devem ser sempre adicionados no seu formato completo
na fila
(c) Levar em consideração páginas html mal-formadas. Você poderá usar
uma API para isso.
(d) Levar em consideração o encoding da página. A classe ColetorUtil pode auxiliar
neste processo
(e) Caso a página não exista, a mesma é ignorada
Você deverá fazer um coletor que obedeça no mínimo os seguintes requisitos:
1) Obedecer os protocolos de exclusão de robôs:
1
Ver plano didático para detalhes sobre a pontuação(a) critérios pertencentes no robots.txt
- Pode ser usado uma API para isto
(b) Critérios “noindex” e “nofolow” das metatags de cada html extraído
ps: noindex: não é permitido coletar. Nofolow: não é permitido seguir os links por
meio desta página
(c) Obedecer o prazo de, no mínimo, 30 segundos entre requisições em um mesmo
servidor (hostname)
→ Caso não obedeça esses critérios o grupo perderá 5 pontos
2) Criar um nome no “User agent” (finalizando como bot) e uma página pessoal com
descrição do coletor, nome dos membros dos grupos, datas das coletas e propósito das
coletas além de um e-mail de contato. Deverá ser explicitado que este coletor baixa
apenas páginas públicas sempre levando em consideração a política de exclusão de
robôs (robots.txt). Esta página deverá ficar online durante o semestre todo. O endereço
da página pessoal deverá ser definida no User agent. Exemplo: “meuBot
(wordpress.org/infoMeuBot)”.
→ O grupo perderá 5 pontos caso não crie a página e/ou não utilize o nome no “User
Agent” devidamente e de acordo com o especificado acima
→ Exemplo de páginas de informação de robôs:
https://support.apple.com/en-us/HT204683
2) Utilizar os seguintes parâmetros no coletor:
→ Número máximo de páginas (50.000 páginas)
→ Profundidade por domínio (6 páginas)
→ Número de threads utilizado
3) Utilizar as sementes de acordo com os integrantes do grupo, usando tabela de
sementes que está apresentado no Moodle em documento separado
4) Produzir um relatório em jupyter a ser definido na próxima seção
5) Armazenar a lista de URLs coletadas



Bibliotecas utilizadas
Vocês poderão usar bibliotecas para os seguintes propósitos (segue também algumas
sugestões de APIs)
→ Extração dos links de páginas HTML mal-formadas:
→ Parser do protocolo de exclusão de robôs
Critério de avaliação
Para a entrega final, o trabalho será avaliado considerando:
1) Legibilidade, comentários e organização do código
2) Funcionamento do coletor
3) Uso do protocolo de exclusão de robôs4) Página de informação do coletor
5) Lista das páginas coletadas
6) Conteúdo do relatório com todos os itens requisitados
Caso não seja feito (ou feito incorretamente) o item (3) e/ou (4) o grupo perderá 5 pontos. O conteúdo do
relatório é tão importante quanto o funcionamento do coletor.
Plágio não será tolerado e, caso identificado, o grupo (quem forneceu e quem utilizou) terá seu
trabalho zerado.